In [1]:
import pyforest
import SimpleITK as sitk
import pydicom
from numba import jit

import sys; sys.path.insert(0, '..')
from database.dataset import *
from models.bria2014.feature_extraction import *

### Preprocessing

#### Check mayimum value in images

In [ ]:
# dcm_path = Path('/home/jseia/Desktop/ml-dl/calc-det/data/INbreast Release 1.0/AllDICOMs/')

# max_val = []
# for dcm in dcm_path.iterdir():
#     if dcm.name.endswith('.dcm'):
#         img = sitk.ReadImage(str(dcm))
#         img_array = sitk.GetArrayFromImage(img)
#         max_val.append(img_array.max())

In [ ]:
# dcm_path = Path('/home/jseia/Desktop/ml-dl/calc-det/data/INbreast Release 1.0/AllDICOMs/')

# bits_alloc = []
# bits_stored = []
# for dcm in dcm_path.iterdir():
#     if dcm.name.endswith('.dcm'):
#         img = pydicom.dcmread(str(dcm), stop_before_piyels=True)
#         bits_alloc.append(img['BitsAllocated'].value)
#         bits_stored.append(img['BitsStored'].value)
#         break

#### Quatum noise tests

In [ ]:
# def T(y, y_max=4095, T_max=65.535):
#     return T_max*np.sqrt(y)/np.sqrt(y_max)

### Detection stage

In [ ]:
# db = INBreast_Dataset(
#     return_lesions_mask=True,
#     level='rois',
#     max_lesion_size_mm=1.0,
#     extract_patches=True,
#     extract_patches_method='all',  # 'centered'
#     patch_size=12,
#     stride=1,
#     min_breast_fraction_roi=0.5,
#     normalize=None,
#     n_jobs=-1,
#     lesion_types=['calcification','normal'],
# )

### feature set

In [2]:
test_image = np.array([
    [5, 3, 4, 1, 2], 
    [5, 3, 4, 1, 2],
    [2, 1, 3, 4, 2],
    [3, 6, 4, 5, 5],
    [4, 3, 2, 6, 1]])
test_integral = to_diag_integral(test_image)

Test the original matrices for an example

In [7]:
# test_Diamond()
# test_FeatureRot()
test_FeatureRot2h()

AssertionError: test Features2h failed: -5 == 50

In [4]:
test_image = np.array([
    [5, 2, 3, 4, 1], 
    [1, 5, 4, 2, 3],
    [2, 2, 1, 3, 4],
    [3, 5, 6, 4, 5],
    [4, 1, 3, 2, 6]])
test_integral = to_diag_integral(test_image)

test_integral


def test_FeatureRot2h():
    expected = Diamond(0, 0, 3, 0.5)(test_integral) - Diamond(2, 2, 3, 0.5)(test_integral)
    actual = Feature2hRot(0, 0, 5, 0.5)(test_integral)
    assert expected == actual, f'test Features2h failed: {expected} == {actual}'


In [6]:
class Feature2hRot(Feature):
    def __init__(self, x: int, y: int, z: int, r: float):  # r es el ratio height/width
        super().__init__(x, y, z, z)

        w = int(z * r)
        assert w // 2 != 0, 'Wrong size of SE'
        f = int(w / 2)

        [Ax, Ay] = [0, z - 2 * f]
        [Bx, By] = [z - 2 * f, 0]
        [Cx, Cy] = [z - f, f]
        [Dx, Dy] = [z, 2 * f]
        [Ex, Ey] = [2 * f, z]
        [Fx, Fy] = [f, z - f]

        self.coords_x = np.array([
            x + Fx, x + Ax, x + Cx, x + Bx,
            x + Ex, x + Fx, x + Dx, x + Cx,
        ])
        self.coords_y = np.array([
            y + Fy - 1, y + Ay - 1, y + Cy - 1, y + By - 1,
            y + Ey - 1, y + Fy - 1, y + Dy - 1, y + Cy - 1,
        ])
        self.coords_x[[1, 3]] = self.coords_x[[1, 3]] - 1
        self.coords_y[[1, 3]] = self.coords_y[[1, 3]] - 1
        self.coeffs   = [
            1,     -1,         -1,          1,
            -1,     1,          1,         -1
        ]


class Feature2vRot(Feature):
    def __init__(self, x: int, y: int, z: int, r: float):  # r es el ratio height/width
        super().__init__(x, y, z, z)

        h = int(z * r)
        assert h // 2 != 0, 'Wrong size of SE'
        f = int(h / 2)

        [Ax, Ay] = [0, 2*f]
        [Bx, By] = [f, f]
        [Cx, Cy] = [2*f, 0]
        [Dx, Dy] = [z, z - 2*f]
        [Ex, Ey] = [z - f, z - f]
        [Fx, Fy] = [z - 2*f, z]

        self.coords_x = [
            x + Ex, x + Dx, x + Bx, x + Cx,
            x + Fx, x + Ax, x + Ex, x + Bx,
        ]
        self.coords_y = [
            y + Ey - 1, y + Dy - 1, y + By - 1, y + Cy - 1,
            y + Fy - 1, y + Ay - 1, y + Ey - 1, y + By - 1,
        ]
        self.coeffs   = [
            1,     -1,         -1,          1,
            -1,     1,          1,         -1
        ]


class Feature3hRot(Feature):
    def __init__(self, x: int, y: int, z: int, r: float):  # r es el ratio height/width
        super().__init__(x, y, z, z)
        
        w = int(z * r)
        assert w // 3 != 0, 'Wrong size of SE'
        f = int(w / 3)

        [Ax, Ay] = [0, z - 3*f]
        [Bx, By] = [z - 3*f, 0]
        [Cx, Cy] = [z - 2*f, f]
        [Dx, Dy] = [z - f, 2*f]
        [Ex, Ey] = [z, 3*f]
        [Fx, Fy] = [3*f, z]
        [Gx, Gy] = [2*f, z - f]
        [Hx, Hy] = [f, z - 2*f]

        self.coords_x = [
            x + Hx, x + Cx, x + Ax, x + Bx,
            x + Gx, x + Dx, x + Hx, x + Cx,
            x + Fx, x + Ex, x + Gx, x + Dx,
        ]
        self.coords_y = [
            y + Hy - 1, y + Cy - 1, y + Ay - 1, y + By - 1,
            y + Gy - 1, y + Dy - 1, y + Hy - 1, y + Cy - 1,
            y + Fy - 1, y + Ey - 1, y + Gy - 1, y + Dy - 1,
        ]
        self.coeffs   = [
             1,     -1,         -1,          1,
            -1,      1,          1,         -1,
             1,     -1,         -1,          1,
        ]


class Feature3vRot(Feature):
    def __init__(self, x: int, y: int, z: int, r: float):  # r es el ratio height/width
        super().__init__(x, y, z, z)

        h = int(z * r)
        assert h // 3 != 0, 'Wrong size of SE'

        f = int(h / 3)
        [Ax, Ay] = [0, 3*f]
        [Bx, By] = [f, 2*f]
        [Cx, Cy] = [2*f, f]
        [Dx, Dy] = [3*f, 0]
        [Ex, Ey] = [z, z - 3*f]
        [Fx, Fy] = [z - f, z - 2*f]
        [Gx, Gy] = [z - 2*f, z - f]
        [Hx, Hy] = [z - 3*f, z]

        self.coords_x = [
            x + Fx, x + Ex, x + Cx, x + Dx,
            x + Gx, x + Fx, x + Bx, x + Cx,
            x + Hx, x + Ax, x + Gx, x + Bx,
        ]
        self.coords_y = [
            y + Fy - 1, y + Ey - 1, y + Cy - 1, y + Dy - 1,
            y + Gy - 1, y + Fy - 1, y + By - 1, y + Cy - 1,
            y + Hy - 1, y + Ay - 1, y + Gy - 1, y + By - 1,
        ]
        self.coeffs   = [
             1,     -1,         -1,          1,
            -1,      1,          1,         -1,
             1,     -1,         -1,          1,
        ]


class Feature4hRot(Feature):
    def __init__(self, x: int, y: int, z: int, r: float):  # r es el ratio height/width
        super().__init__(x, y, z, z)
        
        h = int(z * r)
        assert h // 4 != 0, 'Wrong size of SE'
        f = int(h / 4)

        [Ax, Ay] = [0, 4*f]
        [Bx, By] = [f, 3*f]
        [Cx, Cy] = [2*f, 2*f]
        [Dx, Dy] = [3*f, f]
        [Ex, Ey] = [4*f, 0]
        [Fx, Fy] = [z, z - 4*f]
        [Gx, Gy] = [z - f, z - 3*f]
        [Hx, Hy] = [z - 2*f, z - 2*f]
        [Ix, Iy] = [z - 3*f, z - f]
        [Jx, Jy] = [z - 4*f, z]

        self.coords_x = [
            x + Gx, x + Dx, x + Fx, x + Ex,
            x + Hx, x + Cx, x + Gx, x + Dx,
            x + Ix, x + Bx, x + Hx, x + Cx,
            x + Jx, x + Ax, x + Ix, x + Bx,
        ]
        self.coords_y = [
            y + Gy - 1, y + Dy - 1, y + Fy - 1, y + Ey - 1,
            y + Hy - 1, y + Cy - 1, y + Gy - 1, y + Dy - 1,
            y + Iy - 1, y + By - 1, y + Hy - 1, y + Cy - 1,
            y + Jy - 1, y + Ay - 1, y + Iy - 1, y + By - 1,
        ]
        self.coeffs   = [
             1,     -1,         -1,          1,
            -1,      1,          1,         -1,
            -1,      1,          1,         -1,
             1,     -1,         -1,          1,
        ]


class Feature4vRot(Feature):
    def __init__(self, x: int, y: int, z: int, r: float):  # r es el ratio height/width
        super().__init__(x, y, z, z)
        
        w = int(z * r)
        assert w // 4 != 0, 'Wrong size of SE'
        f = int(w / 4)

        [Ax, Ay] = [0, z - 4*f]
        [Bx, By] = [z - 4*f, 0]
        [Cx, Cy] = [z - 3*f, f]
        [Dx, Dy] = [z - 2*f, 2*f]
        [Ex, Ey] = [z - f, 3*f]
        [Fx, Fy] = [z, 4*f]
        [Gx, Gy] = [4*f, z]
        [Hx, Hy] = [3*f, z - f]
        [Ix, Iy] = [2*f, z - 2*f]
        [Jx, Jy] = [f, z - 3*f]

        self.coords_x = [
            x + Jx, x + Ax, x + Cx, x + Bx,
            x + Ix, x + Jx, x + Dx, x + Cx,
            x + Hx, x + Ix, x + Ex, x + Dx,
            x + Gx, x + Hx, x + Fx, x + Ex,
        ]
        self.coords_y = [
            y + Jy - 1, y + Ay - 1, y + Cy - 1, y + By - 1,
            y + Iy - 1, y + Jy - 1, y + Dy - 1, y + Cy - 1,
            y + Hy - 1, y + Iy - 1, y + Ey - 1, y + Dy - 1,
            y + Gy - 1, y + Hy - 1, y + Fy - 1, y + Ey - 1,
        ]
        self.coeffs   = [
             1,     -1,         -1,          1,
            -1,      1,          1,         -1,
            -1,      1,          1,         -1,
             1,     -1,         -1,          1,
        ]


class FeatureRombo(Feature):
    def __init__(self, x: int, y: int, z: int, r: float):  # r es el ratio height/width
        super().__init__(x, y, z, z)

        assert z // 4 != 0, 'Wrong size of SE'
        f = int(z / 4)

        [Ax, Ay] = [0, 2*f]
        [Bx, By] = [f, f]
        [Cx, Cy] = [2*f, 0]
        [Dx, Dy] = [3*f, f]
        [Ex, Ey] = [4*f, 2*f]
        [Fx, Fy] = [3*f, 3*f]
        [Gx, Gy] = [2*f, 4*f]
        [Hx, Hy] = [f, 3*f]
        [Ix, Iy] = [2*f, 2*f]

        self.coords_x = [
            x + Ix, x + Bx, x + Dx, x + Cx,
            x + Hx, x + Ax, x + Ix, x + Bx,
            x + Fx, x + Ix, x + Ex, x + Dx,
            x + Gx, x + Hx, x + Fx, x + Ix,
        ]
        self.coords_y = [
            y + Iy - 1, y + By - 1, y + Dy - 1, y + Cy - 1,
            y + Hy - 1, y + Ay - 1, y + Iy - 1, y + By - 1,
            y + Fy - 1, y + Iy - 1, y + Ey - 1, y + Dy - 1,
            y + Gy - 1, y + Hy - 1, y + Fy - 1, y + Iy - 1,
        ]
        self.coeffs   = [
             1,     -1,         -1,          1,
            -1,      1,          1,         -1,
            -1,      1,          1,         -1,
             1,     -1,         -1,          1,
        ]


class FeatureCenterRot(Feature):
    def __init__(self, x: int, y: int, z: int, r: float):  # r es el ratio height/width
        super().__init__(x, y, z, z)

        assert z // 6 != 0, 'Wrong size of SE'
        f = int(z / 6)

        [Ax, Ay] = [0, 3*f]
        [Bx, By] = [f, 2*f]
        [Cx, Cy] = [2*f, f]
        [Dx, Dy] = [3*f, 0]
        [Ex, Ey] = [4*f, f]
        [Fx, Fy] = [5*f, 2*f]
        [Gx, Gy] = [6*f, 3*f]
        [Hx, Hy] = [5*f, 4*f]
        [Ix, Iy] = [4*f, 5*f]
        [Jx, Jy] = [3*f, 6*f]
        [Kx, Ky] = [2*f, 5*f]
        [Lx, Ly] = [f, 4*f]
        [Mx, My] = [f, 2*f]
        [Nx, Ny] = [2*f, f]
        [Ox, Oy] = [3*f, 2*f]
        [Px, Py] = [2*f, 3*f]

        self.coords_x = [
            x + Hx, x + Cx, x + Gx, x + Dx,
            x + Mx, x + Bx, x + Nx, x + Cx,
            x + Px, x + Mx, x + Ox, x + Nx,
            x + Ix, x + Px, x + Hx, x + Ox,
            x + Jx, x + Ax, x + Ix, x + Bx,
        ]
        self.coords_y = [
            y + Hy - 1, y + Cy - 1, y + Gy - 1, y + Dy - 1,
            y + My - 1, y + By - 1, y + Ny - 1, y + Cy - 1,
            y + Py - 1, y + My - 1, y + Oy - 1, y + Ny - 1,
            y + Iy - 1, y + Py - 1, y + Hy - 1, y + Oy - 1,
            y + Jy - 1, y + Ay - 1, y + Iy - 1, y + By - 1,
        ]
        self.coeffs   = [
             1,     -1,         -1,          1,
             1,     -1,         -1,          1,
            -1,      1,          1,         -1,
             1,     -1,         -1,          1,
             1,     -1,         -1,          1,
        ]

In [ ]:
import time

mean = []
for i in range(10000):
    start = time.time()
    to_integral(sample_image)
    mean.append(time.time()-start)

print(f'mean: {np.mean(mean)} +- std: {np.std(mean)}')

mean = []
for i in range(10000):
    start = time.time()
    integral(sample_image)
    mean.append(time.time()-start)

print(f'mean: {np.mean(mean)} +- std: {np.std(mean)}')

mean = []
for i in range(10000):
    start = time.time()
    diag_integral(sample_image)
    mean.append(time.time()-start)

print(f'mean: {np.mean(mean)} +- std: {np.std(mean)}')